<a href="https://colab.research.google.com/github/homa-abedin/BigData/blob/main/Passenger_Count_in_month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9ddf38fea1a00886e9503849acc406cddc82bd90f3da35d7bfab7c56314a8b57
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import SparkContext


In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql.functions import *

In [11]:
import datetime
from pyspark.sql.functions import year, month, dayofmonth

In [5]:
spark =SparkSession.builder.master('local[*]').appName('Passenger Count').getOrCreate()

In [6]:
spark

In [9]:
passenger=spark.read.options(sep=',', header=True,inferSchema=True).csv('/content/lax_passengers_header.csv')

In [10]:
passenger.show(10)

+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|     DataExtractDate|        ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|          Arrival|              Domestic|            490|
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|        Departure|              Domestic|            498|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|          Arrival|              Domestic|            753|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|        Departure|              Domestic|            688|
|05/01/2014 12:00:...|01/01/2006 12:00:...|       Terminal 1|          Arrival|              Domestic|         401535|
|05/01/2014 12:00:...|01/01/2006 12:00:...|     

In [19]:
passenger.schema

StructType([StructField('DataExtractDate', StringType(), True), StructField('ReportPeriod', StringType(), True), StructField('Terminal', StringType(), True), StructField('Arrival_Departure', StringType(), True), StructField('Domestic_International', StringType(), True), StructField('Passenger_Count', IntegerType(), True)])

In [77]:
passengerNewDate=passenger.withColumn( 'newdate',passenger.ReportPeriod.substr(1,10))

In [101]:

df=passengerNewDate.select( date_format(to_date("newdate","MM/dd/yyyy"), "yyyy-MM" ).alias("to_date"),'Terminal' , 'Passenger_Count' )

df.show(10)

+-------+-----------------+---------------+
|to_date|         Terminal|Passenger_Count|
+-------+-----------------+---------------+
|2006-01|Imperial Terminal|            490|
|2006-01|Imperial Terminal|            498|
|2006-01|   Misc. Terminal|            753|
|2006-01|   Misc. Terminal|            688|
|2006-01|       Terminal 1|         401535|
|2006-01|       Terminal 1|         389745|
|2006-01|       Terminal 1|            561|
|2006-01|       Terminal 2|          98991|
|2006-01|       Terminal 2|         163067|
|2006-01|       Terminal 2|          93672|
+-------+-----------------+---------------+
only showing top 10 rows



In [102]:
df.schema

StructType([StructField('to_date', StringType(), True), StructField('Terminal', StringType(), True), StructField('Passenger_Count', IntegerType(), True)])

In [103]:
newdf=df.select('*',year("to_date").alias('year'),month("to_date").alias('month'))

In [104]:
newdf.show(10)

+-------+-----------------+---------------+----+-----+
|to_date|         Terminal|Passenger_Count|year|month|
+-------+-----------------+---------------+----+-----+
|2006-01|Imperial Terminal|            490|2006|    1|
|2006-01|Imperial Terminal|            498|2006|    1|
|2006-01|   Misc. Terminal|            753|2006|    1|
|2006-01|   Misc. Terminal|            688|2006|    1|
|2006-01|       Terminal 1|         401535|2006|    1|
|2006-01|       Terminal 1|         389745|2006|    1|
|2006-01|       Terminal 1|            561|2006|    1|
|2006-01|       Terminal 2|          98991|2006|    1|
|2006-01|       Terminal 2|         163067|2006|    1|
|2006-01|       Terminal 2|          93672|2006|    1|
+-------+-----------------+---------------+----+-----+
only showing top 10 rows



In [115]:
newdf.createOrReplaceTempView('newdf')

In [95]:
spark.sql('select Terminal from newdf where Terminal not in ('"'Imperial Terminal'"', '"'Misc. Terminal'"' ) group by Terminal order by Terminal ').show()

+--------------------+
|            Terminal|
+--------------------+
|          Terminal 1|
|          Terminal 2|
|          Terminal 3|
|          Terminal 4|
|          Terminal 5|
|          Terminal 6|
|          Terminal 7|
|          Terminal 8|
|Tom Bradley Inter...|
+--------------------+



In [116]:
final= spark.sql('select to_date , sum(Passenger_Count) as cnt from newdf where Terminal not in ('"'Imperial Terminal'"', '"'Misc. Terminal'"' )  group by to_date having cnt>5000000 ')

In [117]:
final.show(10)

+-------+-------+
|to_date|    cnt|
+-------+-------+
|2013-05|5557314|
|2009-07|5603652|
|2013-09|5159762|
|2007-05|5256763|
|2010-08|5617610|
|2013-12|5606038|
|2006-07|5919529|
|2013-06|5947409|
|2006-05|5176330|
|2006-04|5085946|
+-------+-------+
only showing top 10 rows



In [118]:
newdf.write.csv('/content/passenger.csv')